In [137]:
# библиотеки
import pandas as pd
import numpy as np
import re

In [138]:
work_link = {
    'elements': r'..\Bandgap prediction\Data\Elements\elements for ML.csv',
    'bandgap': r'D:..\Bandgap prediction\Data\Bandgap oxides and nitrides only.csv',
    'interaction data nitrides': r'..\Bandgap prediction\Data\interaction data nitrides.csv',
    'interaction data oxides': r'D:..\Bandgap prediction\Data\interaction data oxides.csv',
    'oxides and nitrides  for predict': r'D:..\Bandgap prediction\Data\oxides and nitrides  for predict.csv'
}

In [139]:
elements = pd.read_csv(filepath_or_buffer=work_link['elements'], index_col='Symbol')

In [140]:
compound = pd.read_csv(filepath_or_buffer=work_link['bandgap'], sep=';')

In [141]:
interaction_data_nitrides = pd.read_csv(filepath_or_buffer=work_link['interaction data nitrides'], sep=';',index_col='Example')

In [142]:
interaction_data_oxides = pd.read_csv(filepath_or_buffer=work_link['interaction data oxides'], sep=';', index_col='Example')

In [143]:
predicted_data = pd.read_csv(filepath_or_buffer=work_link['oxides and nitrides  for predict'], sep=';', index_col='Compound')

In [144]:
import collections
[i for i in collections.Counter(compound['Compound']).items() if i[1] > 1]

[('Fe2TiO5', 2),
 ('Fe2V4O13', 2),
 ('ZrMo2O8', 2),
 ('BaWO4', 2),
 ('BaMoO4', 2),
 ('MgIn2O4', 2),
 ('BiFeO3', 2),
 ('Ag3VO4', 2),
 ('ZnGeN2', 2)]

In [145]:
compound.Band_gap = compound.Band_gap.map(lambda x: float(re.sub(pattern=',', repl='.', string=x)))

In [146]:
# уникальные соединения
unique_index = pd.unique(compound['Compound'])

# создам df с уникальными соединениями. Занчения ширины запрещенной зоны для соединения у которого несколько показаний, усредняется
data = pd.DataFrame(data=[np.around(a=np.mean(index.values), decimals=4) for index in map(lambda x: compound[compound['Compound'] == x]['Band_gap'], unique_index)],
 columns=['Band_gap'], index=pd.Index(unique_index))
data.head()

,Band_gap
Mn2SiO4,3.25
Mn4SiO7,1.52
Mn7SiO12,3.15
AlSi2O5,9.10
Fe2SiO4,7.80


In [147]:
# регулярное выражение для разложения соединения
reg = r"(([A-Z](?![a-z])+|[A-Z][a-z]+)(\d+|))"
# re.findall(pattern=reg, string='Bi2Te2Se') -- > [('Bi2', 'Bi', '2'), ('Te2', 'Te', '2'), ('Se', 'Se', '')]

# расзбирает соединение 
def fun_dearch_elements(element):
    d = dict()
    s = re.findall(pattern=reg, string=element)
    for i in range(len(s)):
        d['element_{f_1}'.format(f_1=i)] = s[i][1]
        d['number_{f_1}'.format(f_1=i)] = s[i][2]
    return d

In [148]:
print(fun_dearch_elements('Mn2SiO4'))

{'element_0': 'Mn', 'number_0': '2', 'element_1': 'Si', 'number_1': '', 'element_2': 'O', 'number_2': '4'}


In [149]:
ele = [re.findall(pattern=reg, string=i)[j][1] for i in data.index for j in range(len(re.findall(pattern=reg, string=i)))]
print('Все элементы в соединениях: ', ele[:50])


Все элементы в соединениях:  ['Mn', 'Si', 'O', 'Mn', 'Si', 'O', 'Mn', 'Si', 'O', 'Al', 'Si', 'O', 'Fe', 'Si', 'O', 'Y', 'Si', 'O', 'Zr', 'Si', 'O', 'In', 'Si', 'O', 'Zn', 'Si', 'O', 'Mn', 'Co', 'O', 'Mn', 'Co', 'O', 'Mn', 'Al', 'O', 'Fe', 'Mn', 'O', 'Cr', 'Mn', 'O', 'Mn', 'Y', 'O', 'Mn', 'Sn', 'O', 'Ga', 'Mn']


In [150]:
dek_el = set(ele) - set(elements.index)
print('Элементы без данных: ', dek_el)

Элементы без данных:  {'La'}


In [151]:
# создам список соеинений, которые необходимо удвлить из data
del_compaunds = []
for compound in data.index:
    a = re.findall(pattern=reg, string=compound)
    b = [a[i][1] for i in range(len(a))]
    if not set(b).isdisjoint(dek_el):
        del_compaunds.append(compound)

In [152]:
print('Список соединений который необходимо удалить:{f1}'.format(f1=del_compaunds))

Список соединений который необходимо удалить:['LaFeO3', 'LaCoO3']


In [153]:
data.drop(del_compaunds, inplace=True)

In [154]:
elements.head()

,Atomic_Number,NUMBER_OF_Electrons_at_last_orbitale,NUMBER_OF_Electrons_at_before_last_orbitale,NUMBER_OF_electrones_at_last_level,NUMBER_OF_vacancies_at_outer_orbitale,Number_of_active_electrons_at_inner_level,Max_valency,Atomic_Mass,Electronegativity,Atomic_radius,Covalent_radius,Ionization_potential,Electron_affinity,Period,Group,Block
Symbol,,,,,,,,,,,,,,,,
Li,3,1,0,1,1,0,1,7.000000,0.98,145.0,134,5.392,59.6,2,0,0
B,5,1,2,3,5,0,1,10.810000,2.04,98.0,82,8.298,26.7,2,1,1
C,6,2,2,4,4,0,2,12.011000,2.55,77.0,77,11.260,153.9,2,2,1
N,7,3,2,5,3,0,5,14.006700,3.04,92.0,75,14.480,7.0,2,3,1
Na,11,1,0,2,1,0,1,22.989769,0.93,190.0,154,5.139,52.8,3,0,0


In [155]:
fun_dearch_elements('SnS2')

{'element_0': 'Sn', 'number_0': '', 'element_1': 'S', 'number_1': '2'}

In [156]:
for index in data.index:
    if len(re.findall(pattern=reg, string=index)) > 3 or len(re.findall(pattern=reg, string=index)) <3:
        print(True)

In [157]:
for element in elements.columns:
    data.insert(loc=0, column='{f_1}_1'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_0'], element] for comp in data.index])

In [158]:
data.insert(loc=0, column='number_element_1',
            value=[(lambda x: int(fun_dearch_elements(x)['number_0']) if fun_dearch_elements(x)['number_0'] != '' else 1)(comp)
            if 'number_0' in fun_dearch_elements(comp).keys() else 0  for comp in data.index])

In [161]:
# 2 элемент

for element in elements.columns:
    data.insert(loc=0, column='{f_1}_2'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_1'], element] if 'element_1' in fun_dearch_elements(comp).keys() else 0 for comp in data.index])

In [162]:
data.insert(loc=0, column='number_element_2',
            value=[(lambda x: int(fun_dearch_elements(x)['number_1']) if fun_dearch_elements(x)['number_1'] != '' else 1)(comp)
            if 'number_1' in fun_dearch_elements(comp).keys() else 0  for comp in data.index])

In [163]:
data

,number_element_2,Block_2,Group_2,Period_2,Electron_affinity_2,Ionization_potential_2,Covalent_radius_2,Atomic_radius_2,Electronegativity_2,Atomic_Mass_2,...,Electronegativity_1,Atomic_Mass_1,Max_valency_1,Number_of_active_electrons_at_inner_level_1,NUMBER_OF_vacancies_at_outer_orbitale_1,NUMBER_OF_electrones_at_last_level_1,NUMBER_OF_Electrons_at_before_last_orbitale_1,NUMBER_OF_Electrons_at_last_orbitale_1,Atomic_Number_1,Band_gap
Mn2SiO4,1,1,2,3,133.6,8.152,111,132.0,1.90,28.08500,...,1.55,54.938040,7,5,0,2,0,2,25,3.25
Mn4SiO7,1,1,2,3,133.6,8.152,111,132.0,1.90,28.08500,...,1.55,54.938040,7,5,0,2,0,2,25,1.52
Mn7SiO12,1,1,2,3,133.6,8.152,111,132.0,1.90,28.08500,...,1.55,54.938040,7,5,0,2,0,2,25,3.15
AlSi2O5,2,1,2,3,133.6,8.152,111,132.0,1.90,28.08500,...,1.61,26.981538,1,0,5,3,2,1,13,9.10
Fe2SiO4,1,1,2,3,133.6,8.152,111,132.0,1.90,28.08500,...,1.83,55.840000,6,6,0,2,0,2,26,7.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca2ZnN2,1,2,13,4,0.0,9.394,131,138.0,1.65,65.40000,...,1.00,40.080000,2,0,0,2,0,2,20,1.90
CaZn2N2,2,2,13,4,0.0,9.394,131,138.0,1.65,65.40000,...,1.00,40.080000,2,0,0,2,0,2,20,1.90
Li3ScN2,1,2,6,4,18.1,6.561,144,162.0,1.36,44.95591,...,0.98,7.000000,1,0,1,1,0,1,3,2.90
NaSnN,1,1,2,5,107.3,7.344,141,162.0,1.96,118.71000,...,0.93,22.989769,1,0,1,2,0,1,11,2.50


In [164]:
# 3 элемент

for element in elements.columns:
    data.insert(loc=0, column='{f_1}_3'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_2'], element] if 'element_2' in fun_dearch_elements(comp).keys() else 0 for comp in data.index])

data.insert(loc=0, column='number_element_3',
            value=[(lambda x: int(fun_dearch_elements(x)['number_2']) if fun_dearch_elements(x)['number_2'] != '' else 1)(comp)
            if 'number_2' in fun_dearch_elements(comp).keys() else 0  for comp in data.index])

In [165]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 303 entries, Mn2SiO4 to LiMgN
Data columns (total 52 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   number_element_3                               303 non-null    int64  
 1   Block_3                                        303 non-null    int64  
 2   Group_3                                        303 non-null    int64  
 3   Period_3                                       303 non-null    int64  
 4   Electron_affinity_3                            303 non-null    float64
 5   Ionization_potential_3                         303 non-null    float64
 6   Covalent_radius_3                              303 non-null    int64  
 7   Atomic_radius_3                                303 non-null    float64
 8   Electronegativity_3                            303 non-null    float64
 9   Atomic_Mass_3                                  303 

In [166]:
for element in elements.columns:
    predicted_data.insert(loc=0, column='{f_1}_1'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_0'], element] for comp in predicted_data.index])

predicted_data.insert(loc=0, column='number_element_1',
            value=[fun_dearch_elements(comp)['number_0'] if fun_dearch_elements(comp)['number_0'] != '' else 1  for comp in predicted_data.index])

In [167]:
# 2 элемент

for element in elements.columns:
    predicted_data.insert(loc=0, column='{f_1}_2'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_1'], element]
                if 'element_1' in fun_dearch_elements(comp).keys() else 0 for comp in predicted_data.index])

predicted_data.insert(loc=0, column='number_element_2',
            value=[(lambda x: fun_dearch_elements(x)['number_1'] if fun_dearch_elements(x)['number_1'] != '' else 1)(comp)
            if 'number_1' in fun_dearch_elements(comp).keys() else 0  for comp in predicted_data.index])

In [168]:
# 3 элемент

for element in elements.columns:
    predicted_data.insert(loc=0, column='{f_1}_3'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_2'], element]
                if 'element_2' in fun_dearch_elements(comp).keys() else 0 for comp in predicted_data.index])

predicted_data.insert(loc=0, column='number_element_3',
            value=[(lambda x: fun_dearch_elements(x)['number_2'] if fun_dearch_elements(x)['number_2'] != '' else 1)(comp)
            if 'number_2' in fun_dearch_elements(comp).keys() else 0  for comp in predicted_data.index])

In [169]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 303 entries, Mn2SiO4 to LiMgN
Data columns (total 52 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   number_element_3                               303 non-null    int64  
 1   Block_3                                        303 non-null    int64  
 2   Group_3                                        303 non-null    int64  
 3   Period_3                                       303 non-null    int64  
 4   Electron_affinity_3                            303 non-null    float64
 5   Ionization_potential_3                         303 non-null    float64
 6   Covalent_radius_3                              303 non-null    int64  
 7   Atomic_radius_3                                303 non-null    float64
 8   Electronegativity_3                            303 non-null    float64
 9   Atomic_Mass_3                                  303 

In [170]:
interaction_data_nitrides['X-Y'].unique()

array([0], dtype=int64)

In [171]:
data['Atomic_Number_3'].unique()

array([ 8, 53, 16,  7], dtype=int64)

In [172]:
data_oxide = data[data['Atomic_Number_3'] == 8]
target_oxide = data_oxide['Band_gap']
data_oxide.drop(columns='Band_gap', inplace=True)

c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\frame.py:3994: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [173]:
data_nitride = data[data['Atomic_Number_3'] == 7]
target_nitride = data_nitride['Band_gap']
data_nitride.drop(columns='Band_gap', inplace=True)

In [174]:
interaction_data_nitrides

,X,Y,N,X-Y,X-N,Y-N,N-X,N-Y
Example,,,,,,,,
Zn3WN4,3,1,4,0,4,4,3,1
ZVN2,1,1,2,0,4,4,2,2
ScMoN3,1,1,3,0,5,5,2,2
Zn2VN3,2,1,3,0,4,4,2,1
Y3Si6N11,3,6,11,0,8,4,3,2
Y5Si3N9,5,3,9,0,7,4,"3,5","1,5"
FeSi7N10,7,1,10,0,4,4,"2,5","0,5"
Co3Y2N4,3,2,4,0,4,5,3,2
Ga3Ge3N5,3,3,5,0,4,3,2,2


In [50]:
# keys_nitrides = ['{f0}-{f1}-{f2}'.format(f0=i[0], f1=i[1], f2=i[2])
#                  for i in zip(interaction_data_nitrides.X, interaction_data_nitrides.Y, interaction_data_nitrides.N)]
# interaction_data_nitrides['kyes'] = keys_nitrides

In [54]:
# interaction_data_nitrides

In [55]:
# fun_dearch_elements('BaSi7N10')

In [40]:
# all_interaction_data = list(interaction_data_nitrides.index) + list(interaction_data_oxides.index)

In [41]:
# all_compound = list(data.index) + list(predicted_data.index)

In [237]:
# set(all_compound) - set(all_interaction_data)

{'Ag3PO4',
 'Ag3VO4',
 'AgBiO3',
 'AgCoO2',
 'AgGaO2',
 'AgNbO3',
 'AgTaO3',
 'AgVO3',
 'Al2CoN3',
 'Al2CrN3',
 'Al2CuO4',
 'Al2FeN3',
 'Al2FeO4',
 'Al2ZnO4',
 'Al3GaN4',
 'Al5BiN4',
 'AlCoN2',
 'AlCr2N3',
 'AlCrN2',
 'AlCuO2',
 'AlFe2N3',
 'AlFeN2',
 'AlGa3N4',
 'AlGaN2',
 'AlInN2',
 'AlNbO4',
 'AlVO4',
 'AlYO3',
 'AlZn35O36',
 'Ba2MnO3',
 'Ba2ZnN2',
 'Ba3Mn2O8',
 'BaAl2O4',
 'BaFe4O7',
 'BaMn2O3',
 'BaMn2O8',
 'BaMnO3',
 'BaMoO4',
 'BaSb2O6',
 'BaSi7N10',
 'BaSiN2',
 'BaSnO3',
 'BaTiO3',
 'BaWO4',
 'BaZnO2',
 'BaZrO3',
 'BaZrS3',
 'Bi25FeO40',
 'Bi2MoO6',
 'Bi2WO6',
 'Bi4Ti3O12',
 'BiFeO3',
 'BiOI',
 'BiPO4',
 'BiVO4',
 'Ca2Mn2O5',
 'Ca2Mn3O8',
 'Ca2Nb2O7',
 'Ca2Si5N8',
 'Ca2SiO4',
 'Ca2Ta2O7',
 'Ca2ZnN2',
 'Ca3B2N4',
 'Ca4Mn2O7',
 'CaCO3',
 'CaGa2O4',
 'CaMg2N2',
 'CaMnO3',
 'CaMoO4',
 'CaSb2O6',
 'CaSiN2',
 'CaSnO3',
 'CaTiO3',
 'CaV2O6',
 'CaWO4',
 'CaZn2N2',
 'Cd2Ge7O16',
 'Cd2GeO4',
 'Cd2SnO4',
 'CdGa2O4',
 'CdIn2O4',
 'CdMoO4',
 'CdSb2O6',
 'CdSnO3',
 'CdTiO3',
 'CdV2O4',
 'CdW

In [196]:
# model
from sklearn.model_selection import train_test_split

In [198]:
# target = data.Band_gap
# data.drop(columns='Band_gap', inplace=True)

In [199]:
from sklearn.model_selection import train_test_split

In [208]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=26)

In [209]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
poly.fit(X_train)

PolynomialFeatures(degree=2, include_bias=True, interaction_only=False,
                   order='C')

In [210]:
X_poly_train = poly.transform(X_train)
X_poly_test = poly.transform(X_test)

In [211]:
from sklearn.svm import SVR

In [346]:
svr = SVR(C=10.0, degree=5)

In [347]:
svr.fit(X=X_poly_train, y=y_train)

SVR(C=10.0, cache_size=200, coef0=0.0, degree=5, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [348]:
svr.score(X=X_test, y=y_test)

0.3581448697103993

In [326]:
interaction_data_oxides.rename(columns={'O':'Z', 'X-O':'X-Z', 'Y-O':'Y-Z', 'O-Y':'Z-Y', 'O-X':'Z-X'}, inplace=True)

In [325]:
interaction_data_nitrides.rename(columns={'N':'Z', 'X-N':'X-Z', 'Y-N':'Y-Z', 'N-X':'Z-X', 'N-Y':'Z-Y'}, inplace=True)

In [329]:
interaction_data_nitrides

,X,Y,Z,X-Y,X-Z,Y-Z,Z-X,Z-Y
Example,,,,,,,,
Zn3WN4,3,1,4,0,4,4,3,1
ZVN2,1,1,2,0,4,4,2,2
ScMoN3,1,1,3,0,5,5,2,2
Zn2VN3,2,1,3,0,4,4,2,1
Y3Si6N11,3,6,11,0,8,4,3,2
Y5Si3N9,5,3,9,0,7,4,"3,5","1,5"
FeSi7N10,7,1,10,0,4,4,"2,5","0,5"
Co3Y2N4,3,2,4,0,4,5,3,2
Ga3Ge3N5,3,3,5,0,4,3,2,2


In [333]:
kyes_nitrides_X_Z = {'{f0}-{f1}-{f2}'.format(f0=i[0], f1=i[1], f2=i[2]): i[3]
                 for i in zip(interaction_data_nitrides['X'], interaction_data_nitrides['Y'], interaction_data_nitrides['Z'], interaction_data_nitrides['X-Z'])}

In [334]:
kyes_nitrides_X_Z

{'3-1-4': 4,
 '1-1-2': 4,
 '1-1-3': 5,
 '2-1-3': 4,
 '3-6-11': 8,
 '5-3-9': 7,
 '7-1-10': 4,
 '3-2-4': 4,
 '3-3-5': 4,
 '1-3-3': 3,
 '2-5-8': 10,
 '1-4-4': 4,
 '1-2-4': 4,
 '1-2-2': 4}

In [340]:
data_nitride['keys'] = ['{f0}-{f1}-{f2}'.format(f0=i['number_0'] if i['number_0'] != '' else 1,
                                       f1=i['number_1'] if i['number_1'] != '' else 1,
                                       f2=i['number_2'] if i['number_2'] != '' else 1) for i in map(fun_dearch_elements, data_nitride.index)]

c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [342]:
data_nitride['X_Z'] = data_nitride['keys'].map(kyes_nitrides_X_Z)

c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [345]:
data_nitride['X_Z']

BaSi7N10     NaN
Ba2Si5N8    10.0
BaSiN2       4.0
SrTiN2       4.0
Sr2Si5N8    10.0
SrSiN2       4.0
CaSiN2       4.0
Ca2Si5N8    10.0
LiGe2N3      NaN
SrCN2        4.0
Ca3B2N4      4.0
MgGeN2       4.0
ZnGeN2       4.0
MgSiN2       4.0
ZnSiN2       4.0
ZnSnN2       4.0
Zn2SbN3      4.0
Mg3SbN       NaN
MgSnN2       4.0
MgZrN2       4.0
MnSiN2       4.0
MnGeN2       4.0
LiSi2N3      NaN
Li7VN4       NaN
Li3GaN2      NaN
Li3AlN2      NaN
LiZnN        NaN
Ba2ZnN2      NaN
CaMg2N2      4.0
Ca2ZnN2      NaN
CaZn2N2      4.0
Li3ScN2      NaN
NaSnN        NaN
LiMgN        NaN
Name: X_Z, dtype: float64

In [335]:
data_nitride.index

Index(['BaSi7N10', 'Ba2Si5N8', 'BaSiN2', 'SrTiN2', 'Sr2Si5N8', 'SrSiN2',
       'CaSiN2', 'Ca2Si5N8', 'LiGe2N3', 'SrCN2', 'Ca3B2N4', 'MgGeN2', 'ZnGeN2',
       'MgSiN2', 'ZnSiN2', 'ZnSnN2', 'Zn2SbN3', 'Mg3SbN', 'MgSnN2', 'MgZrN2',
       'MnSiN2', 'MnGeN2', 'LiSi2N3', 'Li7VN4', 'Li3GaN2', 'Li3AlN2', 'LiZnN',
       'Ba2ZnN2', 'CaMg2N2', 'Ca2ZnN2', 'CaZn2N2', 'Li3ScN2', 'NaSnN',
       'LiMgN'],
      dtype='object')